In [ ]:
"""
统计用户前一个月的回答情况，测试集的最后一天统计23天，sum再除以23/30；
这些情况包括回答过的收藏数等的总和以及平均。
也包括回答总数。
"""

In [1]:
import pandas as pd
import logging

log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [2]:
base_path = './data'
feature_path = './feature'

In [3]:
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
logging.info("invite %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
logging.info("test %s", test.shape)

[2019-12-01 03:22:19,014] INFO in <ipython-input-3-af34e33feb4f>: invite (9489162, 4)
[2019-12-01 03:22:20,414] INFO in <ipython-input-3-af34e33feb4f>: test (1141683, 3)


In [4]:
def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))


def extract_hour(s):
    return s.apply(lambda x: int(x.split('-')[1][1:]))

train['day'] = extract_day(train['dt'])
train['hour'] = extract_hour(train['dt'])

test['day'] = extract_day(test['dt'])
test['hour'] = extract_hour(test['dt'])

del train['dt'], test['dt']

In [5]:
# 加载问题
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']
del ques['title_t1'], ques['title_t2'], ques['desc_t1'], ques['desc_t2']
logging.info("ques %s", ques.shape)

ques['q_day'] = extract_day(ques['q_dt'])
ques['q_hour'] = extract_hour(ques['q_dt'])

del ques['q_dt']

[2019-12-01 03:23:16,138] INFO in <ipython-input-5-9063dae39e17>: ques (1829900, 3)


In [6]:
# 加载回答
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t')
ans.columns = ['aid', 'qid', 'uid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 'has_img',
               'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
               'reci_xxx', 'reci_no_help', 'reci_dis']
del ans['ans_t1'], ans['ans_t2']
logging.info("ans %s", ans.shape)

ans['a_day'] = extract_day(ans['ans_dt'])
ans['a_hour'] = extract_hour(ans['ans_dt'])
del ans['ans_dt']

ans = pd.merge(ans, ques, on='qid', how='left')
del ques

# 回答距提问的天数、总小时数
ans['diff_qa_days'] = ans['a_day'] - ans['q_day']
ans['diff_qa_hours'] = ans['diff_qa_days'] * 24 + (ans['a_hour'] - ans['q_hour'])

[2019-12-01 03:24:35,010] INFO in <ipython-input-6-43a040e1d5dc>: ans (4513735, 18)


In [31]:
extract_feat = ['is_good', 'is_rec', 'is_dest', 'has_img', 'has_video', 'word_count',
              'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
              'reci_xxx', 'reci_no_help', 'reci_dis', 'diff_qa_days', 'diff_qa_hours']
u_group = ans[['uid'] + extract_feat].groupby('uid')

u_sum_stat_raw = u_group[extract_feat].sum()
u_sum_stat_raw.columns = ['u_total_' + feat + '_raw' for feat in extract_feat]

In [32]:
# answer info 里的每个用户的回答数量
u_sum_stat_raw['u_total_answer_raw'] = u_group['is_good'].count()

In [34]:
train = pd.merge(train, ans, on=['qid', 'uid'], how='left')
train = pd.merge(train, u_sum_stat_raw, on='uid', how='left')

In [42]:
# test = pd.merge(test, ans, on=['qid', 'uid'], how='left')
test = pd.merge(test, u_sum_stat_raw, on='uid', how='left')

In [41]:
# 防穿越特征
careful_feat_1 = ['is_good', 'is_rec', 'is_dest', 'has_img', 'has_video', 'word_count', 
                  'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
                  'reci_xxx', 'reci_no_help', 'reci_dis']         # 空值补零
careful_feat_2 = ['diff_qa_days', 'diff_qa_hours']       # 空值不补
careful_feat_3 = ['answer']      # 空值补零，但需要特殊处理

careful_feat = careful_feat_1 + careful_feat_2 + careful_feat_3 

In [75]:
# 有些空值补零， 有的不做处理
for feat in careful_feat_1:
    total_raw_feat = 'u_total_' + feat + '_raw'
    # train
    train[feat] = train[feat].fillna(0)
    train[total_raw_feat] = train[total_raw_feat].fillna(0)
    # test
    test[total_raw_feat] = test[total_raw_feat].fillna(0)

train['u_total_answer_raw'] = train['u_total_answer_raw'].fillna(0)
test['u_total_answer_raw'] = test['u_total_answer_raw'].fillna(0)

train['diff_qa_days'] = train['diff_qa_days'].fillna(0)
train['diff_qa_hours'] = train['diff_qa_hours'].fillna(0)

In [86]:
# 原表防穿越处理
for feat in careful_feat:
    new_name = 'u_total_' + feat
    old_name = 'u_total_' + feat + '_raw'
    
    if feat == 'answer':
        train[new_name] = train[old_name] - train['label']
        train[new_name] = train[new_name].replace(-1, 0)
    else:
        train[new_name] = train[old_name] - train[feat]
    
    test[new_name] = test[old_name]
    
    # 压缩数据
    if feat not in careful_feat_2:
        train[new_name] = train[new_name].astype('int32')
        test[new_name] = test[new_name].astype('int32')
    else:
        train[new_name] = train[new_name].astype('float32')
        test[new_name] = test[new_name].astype('float32')

In [88]:
# 平均
for feat in careful_feat:
    if feat == 'answer':
        continue
    new_name = 'u_mean_' + feat
    old_name = 'u_total_' + feat
    
    train[new_name] = train[old_name] / (train['u_total_answer']+1)   # 伪平均
    test[new_name] = test[old_name] / (test['u_total_answer']+1) 
    
    # 压缩数据
    train[new_name] = train[new_name].astype('float32')
    test[new_name] = test[new_name].astype('float32')

In [113]:
train[train['label']==1][['label', 'u_total_answer_raw', 'u_total_answer']].describe()

,label,u_total_answer_raw,u_total_answer
count,1682918.0,1.682918e+06,1.682918e+06
mean,1.0,1.992352e+01,1.897880e+01
std,0.0,3.423229e+01,3.420087e+01
min,1.0,0.000000e+00,0.000000e+00
25%,1.0,2.000000e+00,1.000000e+00
50%,1.0,7.000000e+00,6.000000e+00
75%,1.0,2.200000e+01,2.100000e+01
max,1.0,9.870000e+02,9.860000e+02


In [114]:
test['u_total_answer'].mean()

10.47682763078718

In [115]:
train['u_total_answer'].mean()

11.288780821741687

In [93]:
test[['u_total_answer','u_total_word_count', 'u_mean_word_count']][:30]

,u_total_answer,u_total_word_count,u_mean_word_count
0,2,2052,684.000000
1,0,0,0.000000
2,4,568,113.599998
3,1,28,14.000000
4,0,0,0.000000
5,2,237,79.000000
6,0,0,0.000000
7,5,315,52.500000
8,7,578,72.250000
9,14,1114,74.266670


In [91]:
train.columns

Index(['qid', 'uid', 'label', 'day', 'hour', 'aid', 'is_good', 'is_rec',
       'is_dest', 'has_img', 'has_video', 'word_count', 'reci_cheer',
       'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks', 'reci_xxx',
       'reci_no_help', 'reci_dis', 'a_day', 'a_hour', 'topic', 'q_day',
       'q_hour', 'diff_qa_days', 'diff_qa_hours', 'u_total_is_good_raw',
       'u_total_is_rec_raw', 'u_total_is_dest_raw', 'u_total_has_img_raw',
       'u_total_has_video_raw', 'u_total_word_count_raw',
       'u_total_reci_cheer_raw', 'u_total_reci_uncheer_raw',
       'u_total_reci_comment_raw', 'u_total_reci_mark_raw',
       'u_total_reci_tks_raw', 'u_total_reci_xxx_raw',
       'u_total_reci_no_help_raw', 'u_total_reci_dis_raw',
       'u_total_diff_qa_days_raw', 'u_total_diff_qa_hours_raw',
       'u_total_answer_raw', 'u_total_is_good', 'u_total_is_rec',
       'u_total_is_dest', 'u_total_has_img', 'u_total_has_video',
       'u_total_word_count', 'u_total_reci_cheer', 'u_total_reci_uncheer

In [94]:
save_feat = ['u_total_is_good', 'u_total_is_rec',
       'u_total_is_dest', 'u_total_has_img', 'u_total_has_video',
       'u_total_word_count', 'u_total_reci_cheer', 'u_total_reci_uncheer',
       'u_total_reci_comment', 'u_total_reci_mark', 'u_total_reci_tks',
       'u_total_reci_xxx', 'u_total_reci_no_help', 'u_total_reci_dis',
       'u_total_diff_qa_days', 'u_total_diff_qa_hours', 'u_total_answer']
save_feat += ['u_mean_is_good', 'u_mean_is_rec', 'u_mean_is_dest', 'u_mean_has_img',
       'u_mean_has_video', 'u_mean_word_count', 'u_mean_reci_cheer',
       'u_mean_reci_uncheer', 'u_mean_reci_comment', 'u_mean_reci_mark',
       'u_mean_reci_tks', 'u_mean_reci_xxx', 'u_mean_reci_no_help',
       'u_mean_reci_dis', 'u_mean_diff_qa_days', 'u_mean_diff_qa_hours']
train[save_feat].min()

u_total_is_good            0.000000
u_total_is_rec             0.000000
u_total_is_dest            0.000000
u_total_has_img            0.000000
u_total_has_video          0.000000
u_total_word_count         0.000000
u_total_reci_cheer         0.000000
u_total_reci_uncheer       0.000000
u_total_reci_comment       0.000000
u_total_reci_mark          0.000000
u_total_reci_tks           0.000000
u_total_reci_xxx           0.000000
u_total_reci_no_help       0.000000
u_total_reci_dis           0.000000
u_total_diff_qa_days      -6.000000
u_total_diff_qa_hours   -145.000000
u_total_answer             0.000000
u_mean_is_good             0.000000
u_mean_is_rec              0.000000
u_mean_is_dest             0.000000
u_mean_has_img             0.000000
u_mean_has_video           0.000000
u_mean_word_count          0.000000
u_mean_reci_cheer          0.000000
u_mean_reci_uncheer        0.000000
u_mean_reci_comment        0.000000
u_mean_reci_mark           0.000000
u_mean_reci_tks            0

In [110]:
train[['label','u_total_diff_qa_days_raw','u_total_diff_qa_days', 'diff_qa_days', 'u_mean_diff_qa_days']][train['label']==1][-50:]

,label,u_total_diff_qa_days_raw,u_total_diff_qa_days,diff_qa_days,u_mean_diff_qa_days
9488893,1,NaN,NaN,0,NaN
9488903,1,46.0,46.0,0,15.333333
9488906,1,76.0,75.0,1,15.000000
9488911,1,1021.0,1016.0,5,101.599998
9488913,1,3688.0,3679.0,9,87.595238
9488928,1,5113.0,5112.0,1,568.000000
9488933,1,21.0,21.0,0,5.250000
9488944,1,560.0,558.0,2,79.714287
9488951,1,6.0,4.0,2,2.000000
9488953,1,19735.0,19735.0,0,138.006989


In [103]:
train[save_feat].min()

u_total_is_good            0.000000
u_total_is_rec             0.000000
u_total_is_dest            0.000000
u_total_has_img            0.000000
u_total_has_video          0.000000
u_total_word_count         0.000000
u_total_reci_cheer         0.000000
u_total_reci_uncheer       0.000000
u_total_reci_comment       0.000000
u_total_reci_mark          0.000000
u_total_reci_tks           0.000000
u_total_reci_xxx           0.000000
u_total_reci_no_help       0.000000
u_total_reci_dis           0.000000
u_total_diff_qa_days      -6.000000
u_total_diff_qa_hours   -145.000000
u_total_answer             0.000000
u_mean_is_good             0.000000
u_mean_is_rec              0.000000
u_mean_is_dest             0.000000
u_mean_has_img             0.000000
u_mean_has_video           0.000000
u_mean_word_count          0.000000
u_mean_reci_cheer          0.000000
u_mean_reci_uncheer        0.000000
u_mean_reci_comment        0.000000
u_mean_reci_mark           0.000000
u_mean_reci_tks            0

In [102]:
test[save_feat].min()

u_total_is_good            0.00
u_total_is_rec             0.00
u_total_is_dest            0.00
u_total_has_img            0.00
u_total_has_video          0.00
u_total_word_count         0.00
u_total_reci_cheer         0.00
u_total_reci_uncheer       0.00
u_total_reci_comment       0.00
u_total_reci_mark          0.00
u_total_reci_tks           0.00
u_total_reci_xxx           0.00
u_total_reci_no_help       0.00
u_total_reci_dis           0.00
u_total_diff_qa_days      -5.00
u_total_diff_qa_hours   -130.00
u_total_answer             0.00
u_mean_is_good             0.00
u_mean_is_rec              0.00
u_mean_is_dest             0.00
u_mean_has_img             0.00
u_mean_has_video           0.00
u_mean_word_count          0.00
u_mean_reci_cheer          0.00
u_mean_reci_uncheer        0.00
u_mean_reci_comment        0.00
u_mean_reci_mark           0.00
u_mean_reci_tks            0.00
u_mean_reci_xxx            0.00
u_mean_reci_no_help        0.00
u_mean_reci_dis            0.00
u_mean_d

In [95]:
train[save_feat].dtypes

u_total_is_good            int32
u_total_is_rec             int32
u_total_is_dest            int32
u_total_has_img            int32
u_total_has_video          int32
u_total_word_count         int32
u_total_reci_cheer         int32
u_total_reci_uncheer       int32
u_total_reci_comment       int32
u_total_reci_mark          int32
u_total_reci_tks           int32
u_total_reci_xxx           int32
u_total_reci_no_help       int32
u_total_reci_dis           int32
u_total_diff_qa_days     float32
u_total_diff_qa_hours    float32
u_total_answer             int32
u_mean_is_good           float32
u_mean_is_rec            float32
u_mean_is_dest           float32
u_mean_has_img           float32
u_mean_has_video         float32
u_mean_word_count        float32
u_mean_reci_cheer        float32
u_mean_reci_uncheer      float32
u_mean_reci_comment      float32
u_mean_reci_mark         float32
u_mean_reci_tks          float32
u_mean_reci_xxx          float32
u_mean_reci_no_help      float32
u_mean_rec

In [97]:
test[save_feat].dtypes

u_total_is_good            int32
u_total_is_rec             int32
u_total_is_dest            int32
u_total_has_img            int32
u_total_has_video          int32
u_total_word_count         int32
u_total_reci_cheer         int32
u_total_reci_uncheer       int32
u_total_reci_comment       int32
u_total_reci_mark          int32
u_total_reci_tks           int32
u_total_reci_xxx           int32
u_total_reci_no_help       int32
u_total_reci_dis           int32
u_total_diff_qa_days     float32
u_total_diff_qa_hours    float32
u_total_answer             int32
u_mean_is_good           float32
u_mean_is_rec            float32
u_mean_is_dest           float32
u_mean_has_img           float32
u_mean_has_video         float32
u_mean_word_count        float32
u_mean_reci_cheer        float32
u_mean_reci_uncheer      float32
u_mean_reci_comment      float32
u_mean_reci_mark         float32
u_mean_reci_tks          float32
u_mean_reci_xxx          float32
u_mean_reci_no_help      float32
u_mean_rec

In [98]:
train[save_feat].to_csv(f'{feature_path}/train_ua_feature.txt', index=False, sep='\t')

In [99]:
test[save_feat].to_csv(f'{feature_path}/test_ua_feature.txt', index=False, sep='\t')